In [48]:
import pandas as pd
from ELO_Engine import Elo_Engine
# Solo para el primer año 1928 / 1929 
def Calculate_ELO(año1,año2):


    route =f"CLEAN_Ligas_Resultados_CSV/laliga_{año1}_{año2}_clean.csv"
    df = pd.read_csv(route)

    equipos = pd.concat([df["Local"], df["Visitante"]]).unique()
    elo_actual = {equipo: 1500 for equipo in equipos}

    historial = []

    for jor in sorted(df["Jornada"].unique()):
        partidos = df[df["Jornada"] == jor]
        for _, row in partidos.iterrows():
            local = row["Local"]
            visitante = row["Visitante"]
            ganador = row["Ganador"]
            diff_goles = row["Differencia Gol"]

            elo_local, elo_visitante = Elo_Engine(
                local,
                visitante,
                ganador,
                elo_actual[local],
                elo_actual[visitante],
                diff_goles
            )

            elo_actual[local] = elo_local
            elo_actual[visitante] = elo_visitante

        for equipo, elo in elo_actual.items():
            historial.append({
                "Jornada": jor,
                "Equipo": equipo,
                "ELO": elo
            })

    elo_df = pd.DataFrame(historial)
    elo_df.to_csv(f"ELO/ELO_{año1}_{año2}.csv", index=False)

Calculate_ELO(1928,1929)

In [50]:
import pandas as pd
from ELO_Engine import Elo_Engine
import os

def calcular_elo_temporada(año1, año2):
    # Saltar años no jugados
    if (año1, año2) in [(1936,1937), (1937,1938), (1938,1939), (1985,1986)]:
        print(f"Año {año1}/{año2} saltado")
        return

    ruta_csv = f"CLEAN_Ligas_Resultados_CSV/laliga_{año1}_{año2}_clean.csv"
    if not os.path.exists(ruta_csv):
        print(f"Archivo no encontrado: {ruta_csv}")
        return

    df = pd.read_csv(ruta_csv)

    carpeta_elo = "ELO"
    os.makedirs(carpeta_elo, exist_ok=True)

    # Cargar ELO anterior si existe
    elo_actual = {}
    if año1 > 1928:
        ruta_elo_prev = os.path.join(carpeta_elo, f"ELO_{año1-1}_{año2-1}.csv")
        if os.path.exists(ruta_elo_prev):
            elo_prev_df = pd.read_csv(ruta_elo_prev)
            max_jornada = elo_prev_df["Jornada"].max()
            base_prev = elo_prev_df[elo_prev_df["Jornada"] == max_jornada]
            elo_actual = dict(zip(base_prev["Equipo"], base_prev["ELO"]))

            # Penalizar equipos que no están en primera este año
            equipos_prev = set(base_prev["Equipo"])
            equipos_actuales = set(pd.concat([df["Local"], df["Visitante"]]).unique())
            equipos_fuera = equipos_prev - equipos_actuales
            for eq in equipos_fuera:
                elo_actual[eq] -= 25  # restar 25 puntos por cada año fuera de primera

    # Inicializar nuevos equipos
    equipos = pd.concat([df["Local"], df["Visitante"]]).unique()
    for equipo in equipos:
        if equipo not in elo_actual:
            elo_actual[equipo] = 1500  # base para equipos nuevos

    historial = []

    for jor in sorted(df["Jornada"].unique()):
        partidos = df[df["Jornada"] == jor]
        for _, row in partidos.iterrows():
            local = row["Local"]
            visitante = row["Visitante"]
            ganador = row["Ganador"]
            diff_goles = row["Differencia Gol"]

            elo_local, elo_visitante = Elo_Engine(
                local,
                visitante,
                ganador,
                elo_actual[local],
                elo_actual[visitante],
                diff_goles
            )

            elo_actual[local] = elo_local
            elo_actual[visitante] = elo_visitante

        # Guardar historial de la jornada
        for equipo, elo in elo_actual.items():
            historial.append({
                "Jornada": jor,
                "Equipo": equipo,
                "ELO": elo
            })

    elo_df = pd.DataFrame(historial)
    elo_df.to_csv(f"{carpeta_elo}/ELO_{año1}_{año2}.csv", index=False)
    print(f"ELO calculado {año1}/{año2}")
    print(elo_df.head())

# Calcular todas las temporadas automáticamente
for año1 in range(1928, 2025):
    año2 = año1 + 1
    calcular_elo_temporada(año1, año2)


ELO calculado 1928/1929
   Jornada           Equipo      ELO
0        1  Arenas de Getxo  1485.58
1        1         Espanyol  1514.42
2        1      Real Madrid  1537.27
3        1    Real Sociedad  1500.00
4        1      Real Racing  1477.15
ELO calculado 1929/1930
   Jornada           Equipo      ELO
0        1  Arenas de Getxo  1439.38
1        1         Espanyol  1431.61
2        1      Real Madrid  1552.62
3        1    Real Sociedad  1480.32
4        1      Real Racing  1448.99
ELO calculado 1930/1931
   Jornada           Equipo      ELO
0        1  Arenas de Getxo  1541.47
1        1         Espanyol  1533.01
2        1      Real Madrid  1550.98
3        1    Real Sociedad  1452.08
4        1      Real Racing  1393.97
ELO calculado 1931/1932
   Jornada           Equipo      ELO
0        1  Arenas de Getxo  1523.48
1        1         Espanyol  1477.54
2        1      Real Madrid  1529.17
3        1    Real Sociedad  1485.08
4        1      Real Racing  1571.21
ELO calculado 19